In [1]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import math, os
from itertools import product
from multiprocessing import Pool
from functions import*
import itertools
import numpy as np

import time
import torch.multiprocessing as mp
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits import mplot3d
import copy
import pdb

In [13]:
X_DIM  = 4
H1_DIM = 128
H2_DIM = 512
REC_DIM = 2
OFF_DIM = 1
V_DIM = 1
LR = 0.0001
EPISODES = 100000
DEADLINE = 20
DISCOUNTS = list(np.linspace(0,1,21))
torch.manual_seed(7)

class boulware(object):
    def __init__(self,V,T, c,r):
        self.V = V
        self.T = T
        self.c = c
        self.r = r
    
    def calc_Ft(self,t):
        return (t/self.T)**(1/self.c)

    def calc_decision_util(self,t):
        Ft = self.calc_Ft(t)
        ut = self.r + (1 - self.r)*(1-Ft)
        return ut
    
    def generate_offer(self,t):
        # How to decide which axis to concede.
        ut = self.calc_decision_util(t)
        ut *= torch.sum(self.V)
        
        X = torch.clamp((ut / V[0]) + np.random.normal(scale=0.05) ,0,1)
        if X == 1:
            ut -= X*V[0]
            Y = torch.clamp((ut / V[1])+ np.random.normal(scale=0.05) ,0,1)
            if Y == 1:
                ut -= Y*V[1]
                Z = torch.clamp((ut / V[2])+ np.random.normal(scale=0.05) ,0,1)
            else:
                Z = 0
        else:
            Y = Z = 0
        
#         print(torch.Tensor([X,Y,Z]))
        return torch.Tensor([X,Y,Z,t])

    def receive(self,offer,t):
        my_offer = 1-offer
        ut = self.calc_decision_util(t)
        ut *= torch.sum(self.V)
        if torch.sum(self.V * my_offer) > ut:
            return torch.Tensor([1])
        else:
            return torch.Tensor([0])


class ON_Cauchy(nn.Module):
    def __init__(self):
        super(ON_Cauchy, self).__init__()
        
        self.V = torch.Tensor([[1,2,3]])
        self.name = "CAUCHY"

        self.L1 = nn.Linear(X_DIM,H1_DIM)

        self.base = nn.Sequential(
            nn.Linear(H1_DIM, H2_DIM),
            nn.ReLU6(),
#             nn.Linear(H2_DIM, H2_DIM),
#             nn.ReLU6(),
        )

        self.muX = nn.Sequential(
            nn.Linear(H1_DIM, H2_DIM),
            nn.ReLU6(),
            nn.Linear(H2_DIM, H2_DIM),
            nn.ReLU6(),
            nn.Linear(H2_DIM, OFF_DIM),
            nn.Sigmoid()
        )

        self.sigX = nn.Sequential(
            nn.Linear(H1_DIM, H2_DIM),
            nn.ReLU6(),
            nn.Linear(H2_DIM, OFF_DIM),
            nn.Sigmoid()
        )
        
        self.muY = nn.Sequential(
            nn.Linear(H1_DIM, H2_DIM),
            nn.ReLU6(),
            nn.Linear(H2_DIM, H2_DIM),
            nn.ReLU6(),
            nn.Linear(H2_DIM, OFF_DIM),
            nn.Sigmoid()
        )

        self.sigY = nn.Sequential(
            nn.Linear(H1_DIM, H2_DIM),
            nn.ReLU6(),
            nn.Linear(H2_DIM, OFF_DIM),
            nn.Sigmoid()
        )
        
        self.muZ = nn.Sequential(
            nn.Linear(H1_DIM, H2_DIM),
            nn.ReLU6(),
           nn.Linear(H2_DIM, H2_DIM),
            nn.ReLU6(),
            nn.Linear(H2_DIM, OFF_DIM),
            nn.Sigmoid()
        )

        self.sigZ = nn.Sequential(
            nn.Linear(H1_DIM, H2_DIM),
            nn.ReLU6(),
            nn.Linear(H2_DIM, OFF_DIM),
            nn.Sigmoid()
        )

        self.m_rec = torch.distributions.Categorical
        self.m_X = torch.distributions.cauchy.Cauchy
        
        
        self.m_Y = torch.distributions.cauchy.Cauchy
        self.m_Z = torch.distributions.cauchy.Cauchy

        self.value = nn.Sequential(
            nn.Linear(H2_DIM, H2_DIM),
            nn.ReLU6(),
            nn.Linear(H2_DIM, H2_DIM),
            nn.ReLU6(),
#             nn.Linear(H2_DIM, H2_DIM),
#             nn.ReLU6(),
            nn.Linear(H2_DIM, V_DIM),
        )

        set_init([self.base, self.muX, self.sigX, self.muY, self.sigY, 
                  self.muZ, self.sigZ, self.value])

    def forward(self, x):
#         out, __ = self.lstm(x)
        out  = self.L1(x)
        muX  = torch.clamp(self.muX(out), 0.02, 0.98)
        sigX = torch.clamp(self.sigX(out), 0.001, 0.02)
        muY  = torch.clamp(self.muY(out), 0.02, 0.98)
        sigY = torch.clamp(self.sigY(out), 0.001, 0.02)
        muZ  = torch.clamp(self.muZ(out), 0.02, 0.98)
        sigZ = torch.clamp(self.sigZ(out), 0.001, 0.02)
        out     = self.base(out)
        return muX, sigX, muY, sigY, muZ, sigZ, self.value(out)
    # Decision, offer, value

    def choose_action(self,x,show=False):
        muX, sigX, muY, sigY, muZ, sigZ, __= self.forward(x)

        ### Select offer ###
        distX = self.m_X(muX.squeeze(), sigX.squeeze())
        distY = self.m_Y(muY.squeeze(), sigY.squeeze())
        distZ = self.m_Z(muZ.squeeze(), sigZ.squeeze())
        X, Y, Z = distX.rsample(), distY.rsample(), distZ.rsample()      
        X, Y, Z = torch.clamp(X, 0.10, 1), torch.clamp(Y, 0.10, 1), torch.clamp(Z, 0.10, 1)
        offer = torch.Tensor([X,Y,Z])

        if show:
            print("Logit", logits)
            print("X: ", muX, sigX)
            print("Y: ", muY, sigY)
            print("Z: ", muZ, sigZ)
        return offer
    
class ON_Normal(nn.Module):
    def __init__(self):
        super(ON_Normal, self).__init__()
        
        self.V = torch.Tensor([[1,2,3]])
        self.name = "NORMAL"

        self.lstm = nn.Linear(X_DIM,H1_DIM)

        self.base = nn.Sequential(
            nn.Linear(H1_DIM, H2_DIM),
            nn.ReLU6(),
#             nn.Linear(H2_DIM, H2_DIM),
#             nn.ReLU6(),
        )

        self.muX = nn.Sequential(
            nn.Linear(H1_DIM, H2_DIM),
            nn.ReLU6(),
            nn.Linear(H2_DIM, H2_DIM),
            nn.ReLU6(),
            nn.Linear(H2_DIM, OFF_DIM),
            nn.Sigmoid()
        )

        self.sigX = nn.Sequential(
            nn.Linear(H1_DIM, H2_DIM),
            nn.ReLU6(),
            nn.Linear(H2_DIM, OFF_DIM),
            nn.Sigmoid()
        )
        
        self.muY = nn.Sequential(
            nn.Linear(H1_DIM, H2_DIM),
            nn.ReLU6(),
            nn.Linear(H2_DIM, H2_DIM),
            nn.ReLU6(),
            nn.Linear(H2_DIM, OFF_DIM),
            nn.Sigmoid()
        )

        self.sigY = nn.Sequential(
            nn.Linear(H1_DIM, H2_DIM),
            nn.ReLU6(),
            nn.Linear(H2_DIM, OFF_DIM),
            nn.Sigmoid()
        )
        
        self.muZ = nn.Sequential(
            nn.Linear(H1_DIM, H2_DIM),
            nn.ReLU6(),
           nn.Linear(H2_DIM, H2_DIM),
            nn.ReLU6(),
            nn.Linear(H2_DIM, OFF_DIM),
            nn.Sigmoid()
        )

        self.sigZ = nn.Sequential(
            nn.Linear(H1_DIM, H2_DIM),
            nn.ReLU6(),
            nn.Linear(H2_DIM, OFF_DIM),
            nn.Sigmoid()
        )

        self.m_rec = torch.distributions.Categorical
        self.m_X = torch.distributions.Normal
        self.m_Y = torch.distributions.Normal
        self.m_Z = torch.distributions.Normal

        self.value = nn.Sequential(
            nn.Linear(H2_DIM, H2_DIM),
            nn.ReLU6(),
            nn.Linear(H2_DIM, H2_DIM),
            nn.ReLU6(),
            nn.Linear(H2_DIM, H2_DIM),
            nn.ReLU6(),
            nn.Linear(H2_DIM, V_DIM),
        )

        set_init([self.base, self.muX, self.sigX, self.muY, self.sigY, 
                  self.muZ, self.sigZ, self.value])

    def forward(self, x):
#         out, __ = self.lstm(x)
        out  = self.lstm(x)
        muX  = torch.clamp(self.muX(out), 0.02, 0.98)
        sigX = torch.clamp(self.sigX(out), 0.001, 0.02)
        muY  = torch.clamp(self.muY(out), 0.02, 0.98)
        siagY = torch.clamp(self.sigY(out), 0.001, 0.02)
        muZ  = torch.clamp(self.muZ(out), 0.02, 0.98)
        sigZ = torch.clamp(self.sigZ(out), 0.001, 0.02)
        out     = self.base(out)
        return muX, sigX, muY, sigY, muZ, sigZ, self.value(out)
    # Decision, offer, value

    def choose_action(self,x,show=False):
        muX, sigX, muY, sigY, muZ, sigZ, __= self.forward(x)

        ### Select offer ###
        distX = self.m_X(muX.squeeze(), sigX.squeeze())
        distY = self.m_Y(muY.squeeze(), sigY.squeeze())
        distZ = self.m_Z(muZ.squeeze(), sigZ.squeeze())
        X, Y, Z = distX.rsample(), distY.rsample(), distZ.rsample()      
        X, Y, Z = torch.clamp(X, 0.10, 1), torch.clamp(Y, 0.10, 1), torch.clamp(Z, 0.10, 1)
        offer = torch.Tensor([X,Y,Z])

        if show:
            print("Logit", logits)
            print("X: ", muX, sigX)
            print("Y: ", muY, sigY)
            print("Z: ", muZ, sigZ)
        return offer

class ON_Beta(nn.Module):
    def __init__(self):
        super(ON_Beta, self).__init__()
        
        self.V = torch.Tensor([[1,2,3]])
        self.name = "BETA"
        self.base = nn.Sequential(
            nn.Linear(X_DIM, H1_DIM),
            nn.ReLU(),
#             nn.Linear(H1_DIM, H2_DIM),
#             nn.ReLU(),
#             nn.Linear(H2_DIM, H2_DIM),
#             nn.ReLU(),
        )
        
        self.alphaX = nn.Sequential(
            nn.Linear(H1_DIM, H2_DIM),
            nn.ReLU(),
            nn.Linear(H2_DIM, H2_DIM),
            nn.ReLU(),
            nn.Linear(H2_DIM, OFF_DIM),
            nn.Softplus()
        )
        
        self.betaX = nn.Sequential(
            nn.Linear(H1_DIM, H2_DIM),
            nn.ReLU(),
            nn.Linear(H2_DIM, H2_DIM),
            nn.ReLU(),
            nn.Linear(H2_DIM, OFF_DIM),
            nn.Softplus()
        )
        
        self.alphaY = nn.Sequential(
            nn.Linear(H1_DIM, H2_DIM),
            nn.ReLU(),
            nn.Linear(H2_DIM, H2_DIM),
            nn.ReLU(),
            nn.Linear(H2_DIM, OFF_DIM),
            nn.Softplus()
        )
        
        self.betaY = nn.Sequential(
            nn.Linear(H1_DIM, H2_DIM),
            nn.ReLU(),
            nn.Linear(H2_DIM, H2_DIM),
            nn.ReLU(),
            nn.Linear(H2_DIM, OFF_DIM),
            nn.Softplus()
        )
        
        self.alphaZ = nn.Sequential(
            nn.Linear(H1_DIM, H2_DIM),
            nn.ReLU(),
            nn.Linear(H2_DIM, H2_DIM),
            nn.ReLU(),
            nn.Linear(H2_DIM, OFF_DIM),
            nn.Softplus()
        )
        
        self.betaZ = nn.Sequential(
            nn.Linear(H1_DIM, H2_DIM),
            nn.ReLU(),
            nn.Linear(H2_DIM, H2_DIM),
            nn.ReLU(),
            nn.Linear(H2_DIM, OFF_DIM),
            nn.Softplus()
        )
        
        self.m_X = torch.distributions.beta.Beta
        self.m_Y = torch.distributions.beta.Beta
        self.m_Z = torch.distributions.beta.Beta
        
        self.value = nn.Sequential(
            nn.Linear(H1_DIM, H2_DIM),
            nn.ReLU6(),
            nn.Linear(H2_DIM, H2_DIM),
            nn.ReLU6(),
            nn.Linear(H2_DIM, H2_DIM),
            nn.ReLU6(),
            nn.Linear(H2_DIM, V_DIM),
        )
        
        set_init([self.base, self.alphaX, self.betaX, self.alphaY, self.betaY, 
                  self.alphaZ, self.betaZ, self.value])
        
    def forward(self, x):
        out = self.base(x)
        aX  = torch.clamp(self.alphaX(out), 0, 2000)
        bX = torch.clamp(self.betaX(out), 0, 2000)
        aY  = torch.clamp(self.alphaY(out), 0, 2000)
        bY = torch.clamp(self.betaY(out), 0, 2000)
        aZ  = torch.clamp(self.alphaZ(out), 0, 2000)
        bZ = torch.clamp(self.betaZ(out), 0, 2000)
        return aX, bX, aY, bY, aZ, bZ, self.value(out)
    
    def choose_action(self,x):
        self.training = False
        aX, bX, aY, bY, aZ, bZ, __  = self.forward(x)
        distX = self.m_X(aX.squeeze() + 1, bX.squeeze() + 1)
        distY = self.m_Y(aY.squeeze() + 1, bY.squeeze() + 1)
        distZ = self.m_Z(aZ.squeeze() + 1, bZ.squeeze() + 1)
        X, Y, Z = distX.rsample(), distY.rsample(), distZ.rsample()      
        X, Y, Z = torch.clamp(X, 0.10, 1), torch.clamp(Y, 0.10, 1), torch.clamp(Z, 0.10, 1)
        offer = torch.Tensor([X,Y,Z])
        return offer

def calc_rewards(P_res, actions, own_v, opp_v):
    receiving = 1 - actions
    own_ut = torch.sum(actions   * own_v, dim=1,keepdim=True)   
    opp_ut = torch.sum(receiving * opp_v, dim=1,keepdim=True)   
    rewards = own_ut * (P_res < opp_ut).float()
    return rewards    
    
def train(num_episodes,c,discount,P_res,LR=0.0001):
    torch.manual_seed(7)
    V = torch.Tensor([3,2,1])
    T = 20
    r = 0.0
    gamma = 0.98
    a = boulware(V,T,c,r)
    Net = ON_Normal()
    optimizer = optim.Adam(Net.parameters(),
                               lr=LR)
    losses = np.zeros(num_episodes)
    rewards_aggregate = np.zeros(num_episodes)
    times = np.zeros(num_episodes)
    start = time.time()
    loss = 0

    path = "c{}-d{}".format(c,discount)
    if not os.path.exists(path):
        os.mkdir(path)
        os.mkdir(path+"/data")

    for ep in range(num_episodes):
        if ep % 500 == 0:
            print("Epoch ", ep, "  with loss at ", loss)
            print("Episode Time is: ", time.time() - start)
            start = time.time()
#             output_training_metrics(ep,Net,a,path)

        states_tensor = X
        actions_tensor = Net.choose_action(states_tensor)
        rewards_tensor = calc_rewards(P_res, actions_tensor, Net.V, a.V)

        muX, sigX, muY, sigY, muZ, sigZ, vals = Net(states_tensor)
        # Critic Loss
        td = rewards_tensor - vals
        c_loss = td.pow(2)
        
        m_X = Net.m_X(muX, sigX)
        log_prob = m_X.log_prob(actions_tensor[:,0].unsqueeze(1))
        entropy = 0.5 + 0.5 * math.log(2 * math.pi) + torch.log(m_X.scale)  # exploration
        exp_v = log_prob * td.detach() + 0.005 * entropy ## Take out the entropy if you have some issues
        a_X_loss = -exp_v

        m_Y = Net.m_Y(muY, muY)
        log_prob = m_Y.log_prob(actions_tensor[:,1].unsqueeze(1))
        entropy = 0.5 + 0.5 * math.log(2 * math.pi) + torch.log(m_Y.scale)  # exploration
        exp_v = log_prob * td.detach() + 0.005 * entropy
        a_Y_loss = -exp_v

        m_Z = Net.m_Z(muZ, muZ)
        log_prob = m_Z.log_prob(actions_tensor[:,2].unsqueeze(1))
        entropy = 0.5 + 0.5 * math.log(2 * math.pi) + torch.log(m_Z.scale)  # exploration
        exp_v = log_prob * td.detach() + 0.005 * entropy
        a_Z_loss = -exp_v

        a_loss = a_X_loss + a_Y_loss + a_Z_loss
        
        loss = (c_loss + a_loss).mean()
        losses[ep] = loss

        optimizer.zero_grad()
        a_X_loss.mean().backward(retain_graph = True)
        a_Y_loss.mean().backward(retain_graph = True)
        a_Z_loss.mean().backward(retain_graph = True)
#         a_loss.mean().backward(retain_graph = True)
        c_loss.mean().backward()
        optimizer.step()

    np.save(path + "/c{}-d{}-losses".format(c,discount), losses)
    np.save(path + "/c{}-d{}-times".format(c,discount),  times)
    np.save(path + "/c{}-d{}-rewards".format(c,discount),  rewards_aggregate)
    torch.save(Net.state_dict(),path + ".th")
    return Net, losses

In [11]:
r = 0.0
T = 20
V = torch.Tensor([3,2,1])
c_list = [3.0]
X = []
P_res = []

for c in c_list:
    a = boulware(V,T,c,r)
    for t in range(T):
        state = a.generate_offer(t) # what the agent sees
        uts = a.calc_decision_util(t)
        X.append(state)
        P_res.append(torch.Tensor([uts*6]))
        
P_res = torch.stack(P_res)

In [14]:
Net, losses = train(100,3.0,1.0, P_res)

Epoch  0   with loss at  0
Episode Time is:  0.0


AttributeError: 'list' object has no attribute 'dim'